In [2]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [4]:
%run -i ../../zkstats/core.py

In [5]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [6]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))



In [ ]:
scales = [1]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


verifier_define_calculation(dummy_data_path, ['col_name'],sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2484/1337869238.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


In [8]:
# prover calculates settings, send to verifier
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)



prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":9,"scale_rebase_multiplier":10,"lookup_range":[-3042,45056],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":911,"total_const_size":5,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,9],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_2484/2234710793.py:10: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*X.size()[1])-torch.sum(torch.log(X)))<=X.size()[1]*torch.log(torch.tensor(1.01)), self.w)


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2
spawning module 0


==== setting up ezkl ====


spawning module 2


Time setup: 7.222949981689453 seconds
Theory output:  tensor(47.6981)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 47.697265625
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2305703999912675162, 13621332081612358697, 8265086842419367025, 2630935785179709972]]], 'proof': '2c8466c144c66c0654dcc77df8c71b41b1708171fb60c9d38b49c598ec7f5e131543d8eb178b034158e1523f4e6bade90e8aeec5518ba7b9fc82be37e363f97c19ab1203afa2fedf2cb53d6e337eee77f0f2755125a1e7135386041070a6366215b176ebd946f16c49f4c011716b50f3382c94df5298db2a1ab3adf55315abc21d95491dca429c7a07fe4d0a1f04dfd3d50e32aeca2297401849e9ba711590f612afebbfbdf429f36b5ca765ae5187be766be8c43f80045c046e2efde9a402d6118b2ae8028cd9f8f379b94228b291a97a33a7096f20bffa38abe5b142d6c95e0c1a7f6c755233da2711fb65ff44b86985a5e76f78a454dbc6021f02bb23812722441af3743db1ad43aa22a375ac73e7e7c1fbd4f612ef7c2df041a0575cea46160c6abbd3604c161df529c1f229ec90ecd9adb8b13a94d3c85d1024262fee471f48371bc36078875b1ea3bbb426af3c9b725609c02afcd36f04c792687904f

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

num_inputs:  1
prf instances:  [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [2305703999912675162, 13621332081612358697, 8265086842419367025, 2630935785179709972]]]
proof boolean:  1.0
proof result 1 : 47.697265625
verified
